## GeoCat experimental

In [ ]:
from collections import defaultdict
import importlib
import json
import pickle
import math
import time
from concurrent.futures import ProcessPoolExecutor

import numpy as np
import pandas as pd
import lib.recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
%matplotlib inline

from lib.constants import experiment_constants
import lib.metrics as metrics
from lib.metricscontroller import MetricsController
import lib.geocat.objfunc as gcobjfunc
import lib.cat_utils as cat_utils
import lib.pgc.pgc as pgc
import lib.areamanager

def string_to_array(string):
    if string == '':
        return list()
    return eval(string)

importlib.reload(lib.metricscontroller)
importlib.reload(lib.constants)
importlib.reload(rec)
importlib.reload(gcobjfunc)
importlib.reload(cat_utils)
importlib.reload(lib.metricscontroller)

N=experiment_constants.N
K=experiment_constants.K
CITY=experiment_constants.CITY
METRICS_K=experiment_constants.METRICS_K
print(CITY)

In [ ]:
#Train load
data_checkin_train = pickle.load(open("../data/checkin/train/"+CITY+".pickle","rb"))

#Test load
ground_truth = defaultdict(set)
for checkin in pickle.load(open("../data/checkin/test/"+CITY+".pickle","rb")):
    ground_truth[checkin['user_id']].add(checkin['poi_id'])
#Pois load
poi_coos = {}
poi_cats = {}
for poi_id,poi in pickle.load(open("../data/poi/"+CITY+".pickle","rb")).items():
    poi_coos[poi_id] = tuple([poi['latitude'],poi['longitude']])
    poi_cats[poi_id] = poi['categories']
    
#Social relations load
social_relations = defaultdict(list)
for user_id,friends in pickle.load(open("../data/user/"+CITY+".pickle","rb")).items():
    social_relations[user_id]=friends

user_num = len(social_relations)
poi_num = len(poi_coos)
user_num,poi_num

# Cat Hierarchy load
dict_alias_title,category_tree,dict_alias_depth=cat_utils.cat_structs("../data/categories.json")
undirected_category_tree=category_tree.to_undirected()

# Training matrix create
training_matrix = np.zeros((user_num, poi_num))
for checkin in data_checkin_train:
    training_matrix[checkin['user_id'],checkin['poi_id']]+=1

# poi neighbors load
poi_neighbors=pickle.load(open("../data/neighbor/"+CITY+".pickle","rb"))

In [ ]:
pgc.cmean_dist_pois(poi_coos)

In [ ]:
all_uids = list(range(user_num))
all_lids = list(range(poi_num))

#np.random.shuffle(all_uids)

In [ ]:
fr=open("../data/result/reclist/"+CITY+"_sigir11_top_100.json")
user_predicted={}
user_overall_scores={}

for line in fr:
    obj=json.loads(line)
    user_predicted[obj['user_id']]=obj['predicted']
    user_overall_scores[obj['user_id']]=obj['scores']

len(user_predicted),len(user_overall_scores)

In [ ]:
# pd.read_csv("../data/result/reclist/sigir11_top_100.txt",header=None,sep='\t')[1]

In [ ]:
# user_predict={}
# for index,row in pd.read_csv("../data/result/reclist/sigir11_top_100.txt",header=None).iterrows():
#     user_id=row[0]
#     lid_list=row[1]
#     user_predict[user_id]=eval(lid_list)

In [ ]:
importlib.reload(rec)
importlib.reload(gcobjfunc)
importlib.reload(metrics)
#precision,recall,ild,gc,pr = [],[],[],[],[]

range_K=range(K)

# mcs=list()
# for k in METRICS_K:
#     mcs.append(MetricsController(metrics=['precision','recall','pr','ild','gc','epc'],algorithm='mostpopular_geocat',k=k))





overall_scores=np.zeros(poi_num)

# for cnt, uid in enumerate(all_uids):
def run_geocat(uid):
    if uid in ground_truth:
        
        pois_score=rec.mostpopularnp(training_matrix,uid)
        predicted=list(reversed(np.argsort(pois_score)))[0:N]
        overall_scores=list(reversed(np.sort(pois_score)))[0:N]
        
        actual=ground_truth[uid]
        start_time = time.time()
 
        predicted,overall_scores=gcobjfunc.geocat(uid,training_matrix,predicted,overall_scores,actual,
                         poi_cats,poi_neighbors,K,undirected_category_tree)

        print("uid → %d, time → %fs" % (uid,time.time()-start_time))
        
        predicted=np.array(predicted)[list(reversed(np.argsort(overall_scores)))]
        overall_scores=list(reversed(np.sort(overall_scores)))

        
        return json.dumps({'user_id':uid,'predicted':list(map(int,predicted)),'scores':list(map(float,overall_scores))})+"\n"                                                      
    return None
        
        
#         for i,k in enumerate(METRICS_K):
#             predicted_at_k=predicted[:k]
            
#             precision_val=metrics.precisionk(actual, predicted_at_k)
#             rec_val=metrics.recallk(actual, predicted_at_k)
#             pr_val=metrics.prk(training_matrix[uid],predicted_at_k,poi_neighbors)
#             ild_val=metrics.ildk(predicted_at_k,poi_cats,undirected_category_tree)
#             gc_val=metrics.gck(uid,training_matrix,poi_cats,predicted_at_k)
#             epc_val=metrics.epck(predicted_at_k,actual,uid,training_matrix)
            
#             mcs[i].append_data([precision_val,rec_val,pr_val,ild_val,gc_val,epc_val])
#             print("userid →",uid,end=',')
#             mcs[i].print_metrics()

#,f"prec@{N}: {np.mean(precision)}, rec@{N}: {np.mean(recall)}")
#         print(final_scores)
#         print(rec_list)
#         print(list(reversed(np.sort(final_scores))))
#         print(np.array(rec_list)[list(reversed(np.argsort(final_scores)))])
#         print()
# for i in mcs:
#     i.fwrite_metrics()

executor = ProcessPoolExecutor()

futures =[executor.submit(run_geocat,uid) for uid in all_uids]
results = [future.result() for future in futures]

result_out = open("../data/result/reclist/"+CITY+"_geocat_" + str(K) + ".json", 'w')
for json_string_result in results:
    result_out.write(json_string_result)
result_out.close()